In [1]:
import sys, os, gc
import subprocess
from math import *
import numpy as np
from datetime import *
from pylab import *
import random
from datetime import datetime
import json
import imgaug as ia
import imgaug.augmenters as iaa
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten
from tensorflow.keras.utils import to_categorical

In [2]:
def vgg6(input_shape=(128, 128, 3)):


    model = tf.keras.models.Sequential(name='VGG6')

    model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape, name='conv1'))
    model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', name='conv2'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(tf.keras.layers.Dropout(0.25))

    model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu', name='conv3'))
    model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu', name='conv4'))
    model.add(tf.keras.layers.BatchNormalization(axis = 3, name = 'bn_2'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(4, 4)))
    model.add(tf.keras.layers.Dropout(0.25))

    ## Fully connected layers
    model.add(tf.keras.layers.Flatten())

       
    model.add(tf.keras.layers.Dense(128, activation='relu', name='fc_3'))
    
    model.add(tf.keras.layers.Dense(64, activation='relu', name='fc_4'))

    model.add(tf.keras.layers.Dense(8, activation='relu', name='fc_5'))
    # output layer
    model.add(tf.keras.layers.Dense(1, activation='tanh', name='fc_out2'))

    return model

def esn_shuffle(array, seed=0):
        np.random.seed(seed)
        np.random.shuffle(array)
        return array

class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super(NpEncoder, self).default(obj)


In [3]:
def esn_shuffle(array, seed=None):
        if seed is not None:
            np.random.seed(seed)
        np.random.shuffle(array)
        return array

In [4]:
seq = iaa.Sequential([
    iaa.Sometimes(0.5, iaa.GaussianBlur(sigma=(0, 0.1))),
    iaa.GammaContrast(gamma=(0.97,1.03)),
    iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 3), per_channel=0.5),
    iaa.Add((-5, 5), per_channel=0.5),
    iaa.Multiply((0.8, 1.2), per_channel=0.5),
    iaa.Sometimes(0.1, iaa.Grayscale(alpha=(0.0, 1.0))),
    iaa.Sometimes(0.50, iaa.Grayscale(alpha=1.)),
    iaa.Fliplr(0.5),
    iaa.Flipud(0.5),
    iaa.Affine(
        rotate=(0, 359),
        scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},
        translate_percent={"x": (-0.15, 0.15), "y": (-0.15, 0.15)}, 
        mode=ia.ALL, cval=(0, 255))
    ], random_order=True)


seqGray = iaa.Sequential([
    iaa.GammaContrast(gamma=(0.97,1.03)),
    iaa.Fliplr(0.5),
    iaa.Flipud(0.5),
    iaa.Affine(
        rotate=(0, 359),
        scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},
        translate_percent={"x": (-0.15, 0.15), "y": (-0.15, 0.15)}, 
        mode=ia.ALL, cval=(255, 255))
    ], random_order=True)

In [5]:
%%time 

data = np.load('samples/RGB_128x128_test_000.npz')
images_RGB = data['images'].astype(np.uint8)
labels_RGB = data['labels'] 
pgcIDs_RGB = data['pgcIDs']
N = images_RGB.shape[0]
filter_RGB = np.chararray(N)
filter_RGB[:] = 'c'
data.close() 
del data


data = np.load('samples/g_128x128_test_000.npz')
images_g = data['images'].astype(np.uint8)
labels_g = data['labels'] 
pgcIDs_g = data['pgcIDs']
N = images_g.shape[0]
filter_g = np.chararray(N)
filter_g[:] = 'g'
data.close() 
del data

data = np.load('samples/r_128x128_test_000.npz')
images_r = data['images'].astype(np.uint8)
labels_r = data['labels'] 
pgcIDs_r = data['pgcIDs']
N = images_r.shape[0]
filter_r = np.chararray(N)
filter_r[:] = 'r'
data.close() 
del data

data = np.load('samples/i_128x128_test_000.npz')
images_i = data['images'].astype(np.uint8)
labels_i = data['labels'] 
pgcIDs_i = data['pgcIDs']
N = images_i.shape[0]
filter_i = np.chararray(N)
filter_i[:] = 'i'
data.close() 
del data

CPU times: user 2.29 s, sys: 310 ms, total: 2.6 s
Wall time: 2.6 s


In [6]:
images_gri = np.concatenate((images_g, images_r, images_i))
labels_gri = np.concatenate((labels_g, labels_r, labels_i))
pgcIDs_gri = np.concatenate((pgcIDs_g, pgcIDs_r, pgcIDs_i))
filter_gri = np.concatenate((filter_g, filter_r, filter_i))

N_RGB = len(labels_RGB)
N_gri = len(labels_gri)

In [7]:
ia.seed(100)

indx = esn_shuffle(np.arange(N_gri), seed=200)
images_aug = seqGray(images=images_gri[indx][:N_RGB,:,:,:])
labels_aug = labels_gri[indx][:N_RGB]
pgcIDs_aug = pgcIDs_gri[indx][:N_RGB]
filter_aug = filter_gri[indx][:N_RGB]

n = len(images_aug)
p = int(n/2)
images_aug[:p] = 255 - images_aug[:p]

ia.seed(200)

images_aug = np.concatenate((seq(images=images_RGB),images_aug))
labels_aug = np.concatenate((labels_RGB,labels_aug))
pgcIDs_aug = np.concatenate((pgcIDs_RGB,pgcIDs_aug))
filter_aug = np.concatenate((filter_RGB,filter_aug))

indx = np.arange(len(images_aug))
indx = esn_shuffle(indx, seed=100)
images_test_aug = images_aug[indx]
labels_test_aug = labels_aug[indx]
pgcIDs_test_aug = pgcIDs_aug[indx]
filter_test_aug = filter_aug[indx]

labels_test_aug = 2.*(labels_test_aug-45.)/45. - 1.

In [8]:
batches = esn_shuffle(np.arange(50), seed=50)

for i in range(1,60):
    batches = np.concatenate((batches, esn_shuffle(np.arange(50), seed=i*50))) 

batches, len(batches)

(array([34, 36,  1, ...,  2, 29,  7]), 3000)

In [9]:
zp_dir = 'Uset3_npz/'
ckpt_dir = 'U3_model06_ckpt/'

def trainer(iter, batches, suffix=None):

  if suffix is None:
    suffix = ''

  batchNo = batches[iter]
  if True: 

    vgg_model = vgg6()
    vgg_model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(), metrics=['mse', 'mae'])

    if iter>0:

      vgg_model = vgg6()
      vgg_model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(), metrics=['mse', 'mae'])
      vgg_model.load_weights(ckpt_dir+str(iter-1)+suffix+".ckpt")

      # if batchNo % 2 == 0 : 
      #   for layer in vgg_model.layers[:-1]:
      #     layer.trainable = False

      with open(ckpt_dir+'evalDict'+suffix+'.json') as json_file:
        evalDict = json.load(json_file)
    else:
      evalDict = {}


    print(iter, batchNo)


    batchFile = npzname = '128x128_train_aug_'+'%02d'%(batchNo+1)+'.npz'
    data = np.load(zp_dir + batchFile)
    images_train_aug = data['images'].astype(np.uint8)
    labels_train_aug = data['labels']
    pgcIDs_train_aug = data['pgcIDs']
    data.close() 
    del data

    labels_train_aug = 2.*(labels_train_aug-45.)/45. - 1.
    
    n_epochs=1
    vgg_model.fit(images_train_aug, labels_train_aug, 
                                      epochs=n_epochs, batch_size=64, 
                                      validation_data=(images_test_aug, labels_test_aug),
                                      verbose=1, shuffle=True)
    evalDict[iter] = {} 
    for key in vgg_model.history.history:
      evalDict[iter][key] = vgg_model.history.history[key][0]
    evalDict[iter]["batchNo"] = batchNo
    evalDict[iter]["batchFile"] = batchFile
    
    with open(ckpt_dir+"evalDict"+suffix+".json", "w", encoding ='utf8') as outfile:
      json.dump(evalDict, outfile, allow_nan=True, cls=NpEncoder)
    
    vgg_model.save_weights(ckpt_dir+str(iter)+suffix+".ckpt")

    del vgg_model
    tf.keras.backend.clear_session()
    del images_train_aug 
    del labels_train_aug
    del pgcIDs_train_aug

    gc.collect()

    return evalDict

In [ ]:
for i in range(0, 2000):
    trainer(i, batches)

0 34
282/282 [==============================] - 27s 69ms/step - loss: 0.6043 - mse: 0.6043 - mae: 0.6314 - val_loss: 0.4547 - val_mse: 0.4547 - val_mae: 0.5559
1 36
282/282 [==============================] - 21s 71ms/step - loss: 0.4727 - mse: 0.4727 - mae: 0.5379 - val_loss: 0.2587 - val_mse: 0.2587 - val_mae: 0.4359
2 1
282/282 [==============================] - 19s 66ms/step - loss: 0.2442 - mse: 0.2442 - mae: 0.4092 - val_loss: 0.2379 - val_mse: 0.2379 - val_mae: 0.4154
3 38
282/282 [==============================] - 19s 67ms/step - loss: 0.2257 - mse: 0.2257 - mae: 0.3873 - val_loss: 0.3353 - val_mse: 0.3353 - val_mae: 0.4974
4 8
282/282 [==============================] - 19s 67ms/step - loss: 0.2232 - mse: 0.2232 - mae: 0.3834 - val_loss: 0.2091 - val_mse: 0.2091 - val_mae: 0.3805
5 16
282/282 [==============================] - 20s 67ms/step - loss: 0.2148 - mse: 0.2148 - mae: 0.3716 - val_loss: 0.2386 - val_mse: 0.2386 - val_mae: 0.4074
6 35
282/282 [============================

282/282 [==============================] - 20s 67ms/step - loss: 0.0572 - mse: 0.0572 - mae: 0.1760 - val_loss: 0.0609 - val_mse: 0.0609 - val_mae: 0.1766
52 1
282/282 [==============================] - 19s 67ms/step - loss: 0.0521 - mse: 0.0521 - mae: 0.1692 - val_loss: 0.0521 - val_mse: 0.0521 - val_mae: 0.1643
53 38
282/282 [==============================] - 20s 67ms/step - loss: 0.0541 - mse: 0.0541 - mae: 0.1713 - val_loss: 0.0574 - val_mse: 0.0574 - val_mae: 0.1793
54 8
282/282 [==============================] - 19s 67ms/step - loss: 0.0511 - mse: 0.0511 - mae: 0.1687 - val_loss: 0.0528 - val_mse: 0.0528 - val_mae: 0.1670
55 16
282/282 [==============================] - 20s 67ms/step - loss: 0.0476 - mse: 0.0476 - mae: 0.1602 - val_loss: 0.0572 - val_mse: 0.0572 - val_mae: 0.1735
56 35
282/282 [==============================] - 19s 67ms/step - loss: 0.0497 - mse: 0.0497 - mae: 0.1641 - val_loss: 0.0463 - val_mse: 0.0463 - val_mae: 0.1517
57 13
282/282 [===========================

102 37
282/282 [==============================] - 19s 66ms/step - loss: 0.0390 - mse: 0.0390 - mae: 0.1456 - val_loss: 0.0366 - val_mse: 0.0366 - val_mae: 0.1357
103 28
282/282 [==============================] - 19s 66ms/step - loss: 0.0395 - mse: 0.0395 - mae: 0.1447 - val_loss: 0.0384 - val_mse: 0.0384 - val_mae: 0.1399
104 43
282/282 [==============================] - 19s 66ms/step - loss: 0.0379 - mse: 0.0379 - mae: 0.1437 - val_loss: 0.0462 - val_mse: 0.0462 - val_mae: 0.1556
105 49
282/282 [==============================] - 19s 66ms/step - loss: 0.0396 - mse: 0.0396 - mae: 0.1467 - val_loss: 0.0361 - val_mse: 0.0361 - val_mae: 0.1350
106 5
282/282 [==============================] - 19s 66ms/step - loss: 0.0385 - mse: 0.0385 - mae: 0.1441 - val_loss: 0.0356 - val_mse: 0.0356 - val_mae: 0.1328
107 33
282/282 [==============================] - 19s 66ms/step - loss: 0.0383 - mse: 0.0383 - mae: 0.1432 - val_loss: 0.0371 - val_mse: 0.0371 - val_mae: 0.1373
108 20
282/282 [=============

153 2
282/282 [==============================] - 19s 66ms/step - loss: 0.0328 - mse: 0.0328 - mae: 0.1329 - val_loss: 0.0373 - val_mse: 0.0373 - val_mae: 0.1375
154 16
282/282 [==============================] - 19s 66ms/step - loss: 0.0332 - mse: 0.0332 - mae: 0.1327 - val_loss: 0.0377 - val_mse: 0.0377 - val_mae: 0.1400
155 47
282/282 [==============================] - 19s 66ms/step - loss: 0.0323 - mse: 0.0323 - mae: 0.1322 - val_loss: 0.0366 - val_mse: 0.0366 - val_mae: 0.1349
156 33
282/282 [==============================] - 19s 66ms/step - loss: 0.0333 - mse: 0.0333 - mae: 0.1346 - val_loss: 0.0337 - val_mse: 0.0337 - val_mae: 0.1302
157 32
282/282 [==============================] - 19s 66ms/step - loss: 0.0329 - mse: 0.0329 - mae: 0.1328 - val_loss: 0.0370 - val_mse: 0.0370 - val_mae: 0.1356
158 21
282/282 [==============================] - 19s 66ms/step - loss: 0.0354 - mse: 0.0354 - mae: 0.1377 - val_loss: 0.0346 - val_mse: 0.0346 - val_mae: 0.1297
159 22
282/282 [=============

282/282 [==============================] - 19s 66ms/step - loss: 0.0329 - mse: 0.0329 - mae: 0.1325 - val_loss: 0.0359 - val_mse: 0.0359 - val_mae: 0.1319
204 20
282/282 [==============================] - 19s 66ms/step - loss: 0.0307 - mse: 0.0307 - mae: 0.1291 - val_loss: 0.0353 - val_mse: 0.0353 - val_mae: 0.1334
205 49
282/282 [==============================] - 19s 66ms/step - loss: 0.0299 - mse: 0.0299 - mae: 0.1266 - val_loss: 0.0345 - val_mse: 0.0345 - val_mae: 0.1318
206 18
282/282 [==============================] - 19s 66ms/step - loss: 0.0315 - mse: 0.0315 - mae: 0.1285 - val_loss: 0.0395 - val_mse: 0.0395 - val_mae: 0.1361
207 17
282/282 [==============================] - 19s 66ms/step - loss: 0.0315 - mse: 0.0315 - mae: 0.1294 - val_loss: 0.0377 - val_mse: 0.0377 - val_mae: 0.1376
208 44
282/282 [==============================] - 19s 66ms/step - loss: 0.0325 - mse: 0.0325 - mae: 0.1311 - val_loss: 0.0418 - val_mse: 0.0418 - val_mae: 0.1466
209 32
282/282 [===================

...

282/282 [==============================] - 20s 68ms/step - loss: 0.0169 - mse: 0.0169 - mae: 0.0974 - val_loss: 0.0299 - val_mse: 0.0299 - val_mae: 0.1174
1315 31
282/282 [==============================] - 20s 68ms/step - loss: 0.0172 - mse: 0.0172 - mae: 0.0981 - val_loss: 0.0294 - val_mse: 0.0294 - val_mae: 0.1175
1316 5
282/282 [==============================] - 20s 68ms/step - loss: 0.0169 - mse: 0.0169 - mae: 0.0971 - val_loss: 0.0310 - val_mse: 0.0310 - val_mae: 0.1199
1317 38
282/282 [==============================] - 20s 68ms/step - loss: 0.0171 - mse: 0.0171 - mae: 0.0977 - val_loss: 0.0323 - val_mse: 0.0323 - val_mae: 0.1228
1318 36
282/282 [==============================] - 20s 68ms/step - loss: 0.0169 - mse: 0.0169 - mae: 0.0975 - val_loss: 0.0310 - val_mse: 0.0310 - val_mae: 0.1190
1319 48
282/282 [==============================] - 20s 68ms/step - loss: 0.0176 - mse: 0.0176 - mae: 0.0991 - val_loss: 0.0302 - val_mse: 0.0302 - val_mae: 0.1171
1320 7
282/282 [===============

282/282 [==============================] - 20s 67ms/step - loss: 0.0172 - mse: 0.0172 - mae: 0.0981 - val_loss: 0.0300 - val_mse: 0.0300 - val_mae: 0.1161
1365 3
282/282 [==============================] - 20s 67ms/step - loss: 0.0172 - mse: 0.0172 - mae: 0.0982 - val_loss: 0.0290 - val_mse: 0.0290 - val_mae: 0.1189
1366 49
282/282 [==============================] - 19s 67ms/step - loss: 0.0170 - mse: 0.0170 - mae: 0.0975 - val_loss: 0.0349 - val_mse: 0.0349 - val_mae: 0.1268
1367 9
282/282 [==============================] - 20s 67ms/step - loss: 0.0170 - mse: 0.0170 - mae: 0.0979 - val_loss: 0.0277 - val_mse: 0.0277 - val_mae: 0.1146
1368 10
282/282 [==============================] - 19s 67ms/step - loss: 0.0178 - mse: 0.0178 - mae: 0.0994 - val_loss: 0.0298 - val_mse: 0.0298 - val_mae: 0.1179
1369 1
282/282 [==============================] - 20s 67ms/step - loss: 0.0173 - mse: 0.0173 - mae: 0.0989 - val_loss: 0.0291 - val_mse: 0.0291 - val_mae: 0.1180
1370 34
282/282 [================

282/282 [==============================] - 19s 66ms/step - loss: 0.0167 - mse: 0.0167 - mae: 0.0977 - val_loss: 0.0307 - val_mse: 0.0307 - val_mae: 0.1179
1415 44
282/282 [==============================] - 19s 66ms/step - loss: 0.0169 - mse: 0.0169 - mae: 0.0981 - val_loss: 0.0280 - val_mse: 0.0280 - val_mae: 0.1150
1416 33
282/282 [==============================] - 19s 66ms/step - loss: 0.0166 - mse: 0.0166 - mae: 0.0965 - val_loss: 0.0300 - val_mse: 0.0300 - val_mae: 0.1211
1417 48
282/282 [==============================] - 19s 66ms/step - loss: 0.0171 - mse: 0.0171 - mae: 0.0979 - val_loss: 0.0305 - val_mse: 0.0305 - val_mae: 0.1197
1418 6
282/282 [==============================] - 19s 66ms/step - loss: 0.0164 - mse: 0.0164 - mae: 0.0962 - val_loss: 0.0298 - val_mse: 0.0298 - val_mae: 0.1179
1419 2
282/282 [==============================] - 19s 66ms/step - loss: 0.0175 - mse: 0.0175 - mae: 0.0985 - val_loss: 0.0294 - val_mse: 0.0294 - val_mae: 0.1169
1420 15
282/282 [===============

282/282 [==============================] - 19s 66ms/step - loss: 0.0161 - mse: 0.0161 - mae: 0.0966 - val_loss: 0.0312 - val_mse: 0.0312 - val_mae: 0.1204
1465 38
282/282 [==============================] - 19s 66ms/step - loss: 0.0185 - mse: 0.0185 - mae: 0.1003 - val_loss: 0.0297 - val_mse: 0.0297 - val_mae: 0.1172
1466 14
282/282 [==============================] - 19s 66ms/step - loss: 0.0152 - mse: 0.0152 - mae: 0.0934 - val_loss: 0.0322 - val_mse: 0.0322 - val_mae: 0.1210
1467 19
282/282 [==============================] - 19s 66ms/step - loss: 0.0164 - mse: 0.0164 - mae: 0.0962 - val_loss: 0.0321 - val_mse: 0.0321 - val_mae: 0.1198
1468 41
282/282 [==============================] - 19s 66ms/step - loss: 0.0159 - mse: 0.0159 - mae: 0.0951 - val_loss: 0.0336 - val_mse: 0.0336 - val_mae: 0.1211
1469 30
282/282 [==============================] - 19s 66ms/step - loss: 0.0162 - mse: 0.0162 - mae: 0.0962 - val_loss: 0.0320 - val_mse: 0.0320 - val_mae: 0.1225
1470 46
282/282 [=============

282/282 [==============================] - 19s 66ms/step - loss: 0.0173 - mse: 0.0173 - mae: 0.0968 - val_loss: 0.0364 - val_mse: 0.0364 - val_mae: 0.1323
1515 26
282/282 [==============================] - 19s 66ms/step - loss: 0.0165 - mse: 0.0165 - mae: 0.0975 - val_loss: 0.0305 - val_mse: 0.0305 - val_mae: 0.1179
1516 43
282/282 [==============================] - 19s 66ms/step - loss: 0.0169 - mse: 0.0169 - mae: 0.0976 - val_loss: 0.0294 - val_mse: 0.0294 - val_mae: 0.1164
1517 22
282/282 [==============================] - 19s 66ms/step - loss: 0.0170 - mse: 0.0170 - mae: 0.0972 - val_loss: 0.0293 - val_mse: 0.0293 - val_mae: 0.1170
1518 6
282/282 [==============================] - 19s 66ms/step - loss: 0.0171 - mse: 0.0171 - mae: 0.0980 - val_loss: 0.0324 - val_mse: 0.0324 - val_mae: 0.1242
1519 23
282/282 [==============================] - 19s 67ms/step - loss: 0.0165 - mse: 0.0165 - mae: 0.0967 - val_loss: 0.0289 - val_mse: 0.0289 - val_mae: 0.1164
1520 8
282/282 [===============